# Parquet vs. Vortex vs. Lance performance

Generate a wide random dataset (~100k rows x 4k float columns + 50 string columns) and benchmark on-disk formats. Decrease `N_ROWS`/`N_COLS` if you run into memory pressure.

Requires `pyarrow`, `lancedb`, `vortex-data`, `duckdb` (installed via uv).


**Setup**
- Run `uv run poe lab` (or `uv venv && uv sync && uv run jupyter lab`).
- Artifacts are written under `data/` (git-ignored).
- CSV/DuckDB can be heavy at full size; sampling knobs are provided below.


In [1]:
import gc
import shutil
import time
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import lancedb
import duckdb
import vortex
import vortex.io as vxio

pd.set_option("display.precision", 4)


In [2]:
DATA_DIR = Path("data")
shutil.rmtree(DATA_DIR, ignore_errors=True)
DATA_DIR.mkdir(exist_ok=True)

N_ROWS = 100_000
N_COLS = 4_000  # float columns
STR_COLS = 50
DTYPE = np.float64
REPEATS = 3
SEED = 13

# Downsample for heavy formats to avoid multi-GB outputs.
RUN_CSV = True
RUN_DUCKDB = True
CSV_ROWS = 5_000   # set None to use full table
DUCK_ROWS = 5_000  # set None to use full table
CSV_REPEATS = 1
DUCK_REPEATS = 1

PARQUET_PATH = DATA_DIR / "wide.parquet"
LANCE_PATH = DATA_DIR / "lance_db"
VORTEX_PATH = DATA_DIR / "wide.vortex"
CSV_PATH = DATA_DIR / "wide.csv.gz"
DUCK_PATH = DATA_DIR / "wide.duckdb"
LANCE_TABLE = "bench"
DUCK_TABLE = "bench"

N_ROWS, N_COLS, STR_COLS, DTYPE


(100000, 4000, 50, numpy.float64)

In [3]:
rng = np.random.default_rng(SEED)
float_names = [f"col_{i:04d}" for i in range(N_COLS)]
float_columns = [pa.array(rng.standard_normal(N_ROWS, dtype=DTYPE)) for _ in range(N_COLS)]

str_names = [f"str_{i:04d}" for i in range(STR_COLS)]
str_columns = []
for _ in range(STR_COLS):
    ints = rng.integers(0, 1_000_000, size=N_ROWS, dtype=np.int32)
    strings = np.char.add('s', ints.astype(str))
    str_columns.append(pa.array(strings))

column_names = float_names + str_names
columns = float_columns + str_columns
table = pa.Table.from_arrays(columns, names=column_names)
print('table rows:', table.num_rows, 'cols:', table.num_columns)


table rows: 100000 cols: 4050


In [4]:
# Optional downsampled tables for CSV/DuckDB
csv_table = table.slice(0, CSV_ROWS) if RUN_CSV and CSV_ROWS is not None else table if RUN_CSV else None
duck_table = table.slice(0, DUCK_ROWS) if RUN_DUCKDB and DUCK_ROWS is not None else table if RUN_DUCKDB else None

if RUN_CSV:
    df_csv = csv_table.to_pandas()
    print('CSV sample size (rows, cols):', df_csv.shape)
else:
    df_csv = None
    print('CSV benchmark disabled')


CSV sample size (rows, cols): (5000, 4050)


In [5]:
def drop_path(path: Path) -> None:
    """Delete a file or directory if it exists."""
    if path.is_dir():
        shutil.rmtree(path, ignore_errors=True)
    elif path.exists():
        path.unlink()


def path_size_bytes(path: Path) -> int:
    if path.is_file():
        return path.stat().st_size
    if path.is_dir():
        return sum(p.stat().st_size for p in path.rglob("*") if p.is_file())
    return 0


def run_benchmarks(table: pa.Table, configs, repeats: int = 3):
    results = []
    for cfg in configs:
        cfg_repeats = cfg.get('repeats', repeats)
        write_times = []
        read_times = []
        for run_idx in range(cfg_repeats):
            if cfg.get('cleanup', True):
                drop_path(cfg['path'])
            t0 = time.perf_counter()
            cfg_table = cfg.get('table', table)
            cfg['write'](cfg_table, cfg['path'])
            elapsed = time.perf_counter() - t0
            write_times.append(elapsed)
            print(f"[write] {cfg['name']} run {run_idx + 1}/{cfg_repeats}: {elapsed:.2f}s", flush=True)

        size_bytes = path_size_bytes(cfg['path'])

        for run_idx in range(cfg_repeats):
            gc.collect()
            t0 = time.perf_counter()
            _ = cfg['read'](cfg['path'])
            elapsed = time.perf_counter() - t0
            read_times.append(elapsed)
            print(f"[read ] {cfg['name']} run {run_idx + 1}/{cfg_repeats}: {elapsed:.2f}s", flush=True)

        results.append(
            {
                'format': cfg['name'],
                'write_seconds': write_times,
                'read_seconds': read_times,
                'size_mb': size_bytes / (1024 * 1024),
            }
        )
    return results


In [6]:
# Prepare a single Lance connection ahead of time to avoid connect overhead.
drop_path(LANCE_PATH)
LANCE_DB = lancedb.connect(LANCE_PATH)

def reset_lance_table(db, table_name):
    try:
        if hasattr(db, 'table_names') and table_name in db.table_names():
            if hasattr(db, 'drop_table'):
                db.drop_table(table_name)
            else:
                drop_path(LANCE_PATH)
                return lancedb.connect(LANCE_PATH)
    except Exception:
        drop_path(LANCE_PATH)
        return lancedb.connect(LANCE_PATH)
    return db


In [7]:
def lance_write(tbl, path=LANCE_PATH, table_name=LANCE_TABLE):
    global LANCE_DB
    LANCE_DB = reset_lance_table(LANCE_DB, table_name)
    LANCE_DB.create_table(table_name, tbl, mode="overwrite")


def lance_read(path=LANCE_PATH, table_name=LANCE_TABLE):
    return LANCE_DB.open_table(table_name).to_arrow()


def vortex_write(tbl, path=VORTEX_PATH):
    drop_path(path)
    vxio.write(tbl, str(path))


def vortex_read(path=VORTEX_PATH):
    return vortex.open(str(path)).to_arrow().read_all()


def csv_write(tbl, path=CSV_PATH, df=df_csv):
    drop_path(path)
    df.to_csv(path, index=False, compression='gzip')


def csv_read(path=CSV_PATH):
    return pd.read_csv(path, compression='gzip')


def duck_write(tbl, path=DUCK_PATH, table_name=DUCK_TABLE):
    drop_path(path)
    con = duckdb.connect(str(path))
    con.register('tmp_tbl', tbl)
    con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM tmp_tbl")
    con.close()


def duck_read(path=DUCK_PATH, table_name=DUCK_TABLE):
    con = duckdb.connect(str(path))
    try:
        return con.execute(f"SELECT * FROM {table_name}").arrow()
    finally:
        con.close()


format_configs = [
    {
        'name': 'Parquet (pyarrow, zstd)',
        'path': PARQUET_PATH,
        'write': lambda tbl, path=PARQUET_PATH: pq.write_table(tbl, path, compression='zstd'),
        'read': lambda path=PARQUET_PATH: pq.read_table(path),
    },
    {
        'name': 'Lance (lancedb)',
        'path': LANCE_PATH,
        'write': lance_write,
        'read': lance_read,
        'cleanup': False,
    },
    {
        'name': 'Vortex',
        'path': VORTEX_PATH,
        'write': vortex_write,
        'read': vortex_read,
    },
]

if RUN_CSV:
    format_configs.append(
        {
            'name': 'CSV (pandas, gzip)',
            'path': CSV_PATH,
            'write': csv_write,
            'read': csv_read,
            'table': csv_table,
            'repeats': CSV_REPEATS,
        }
    )

if RUN_DUCKDB:
    format_configs.append(
        {
            'name': 'DuckDB (file table)',
            'path': DUCK_PATH,
            'write': duck_write,
            'read': duck_read,
            'table': duck_table,
            'repeats': DUCK_REPEATS,
        }
    )

print('Formats:', [cfg['name'] for cfg in format_configs])


Formats: ['Parquet (pyarrow, zstd)', 'Lance (lancedb)', 'Vortex', 'CSV (pandas, gzip)', 'DuckDB (file table)']


In [8]:
results = run_benchmarks(table, format_configs, repeats=REPEATS)
results_df = pd.DataFrame(
    {
        'format': [r['format'] for r in results],
        'write_avg_s': [np.mean(r['write_seconds']) for r in results],
        'write_std_s': [np.std(r['write_seconds']) for r in results],
        'read_avg_s': [np.mean(r['read_seconds']) for r in results],
        'read_std_s': [np.std(r['read_seconds']) for r in results],
        'size_mb': [r['size_mb'] for r in results],
    }
)
results_df


[write] Parquet (pyarrow, zstd) run 1/3: 14.85s
[write] Parquet (pyarrow, zstd) run 2/3: 13.64s
[write] Parquet (pyarrow, zstd) run 3/3: 13.69s
[read ] Parquet (pyarrow, zstd) run 1/3: 3.22s
[read ] Parquet (pyarrow, zstd) run 2/3: 2.01s
[read ] Parquet (pyarrow, zstd) run 3/3: 0.97s


[2025-12-02T22:35:46Z WARN  lance::dataset::write::insert] No existing dataset at /Users/buntend/Documents/work/demo-compare-parquet-vortex-lance-perf/notebooks/data/lance_db/bench.lance, it will be created


[write] Lance (lancedb) run 1/3: 6.99s


[2025-12-02T22:35:53Z WARN  lance::dataset::write::insert] No existing dataset at /Users/buntend/Documents/work/demo-compare-parquet-vortex-lance-perf/notebooks/data/lance_db/bench.lance, it will be created


[write] Lance (lancedb) run 2/3: 4.94s


[2025-12-02T22:35:59Z WARN  lance::dataset::write::insert] No existing dataset at /Users/buntend/Documents/work/demo-compare-parquet-vortex-lance-perf/notebooks/data/lance_db/bench.lance, it will be created


[write] Lance (lancedb) run 3/3: 3.59s
[read ] Lance (lancedb) run 1/3: 4.66s
[read ] Lance (lancedb) run 2/3: 5.01s
[read ] Lance (lancedb) run 3/3: 4.98s
[write] Vortex run 1/3: 8.33s
[write] Vortex run 2/3: 6.72s
[write] Vortex run 3/3: 6.74s
[read ] Vortex run 1/3: 1.06s
[read ] Vortex run 2/3: 0.60s
[read ] Vortex run 3/3: 0.59s
[write] CSV (pandas, gzip) run 1/1: 34.78s
[read ] CSV (pandas, gzip) run 1/1: 2.23s
[write] DuckDB (file table) run 1/1: 1.61s
[read ] DuckDB (file table) run 1/1: 0.14s


,format,write_avg_s,write_std_s,read_avg_s,read_std_s,size_mb
0,"Parquet (pyarrow, zstd)",14.0588,0.5571,2.0674,0.9200,3742.9114
1,Lance (lancedb),5.1737,1.3992,4.8847,0.1570,3123.8285
2,Vortex,7.2661,0.7541,0.7511,0.2206,2769.6237
3,"CSV (pandas, gzip)",34.7804,0.0000,2.2322,0.0000,173.8194
4,DuckDB (file table),1.6101,0.0000,0.1362,0.0000,181.0117


In [9]:
timings = []
for r in results:
    for idx, t in enumerate(r['write_seconds']):
        timings.append({'format': r['format'], 'kind': 'write', 'run': idx, 'seconds': t})
    for idx, t in enumerate(r['read_seconds']):
        timings.append({'format': r['format'], 'kind': 'read', 'run': idx, 'seconds': t})

pd.DataFrame(timings)


,format,kind,run,seconds
0,"Parquet (pyarrow, zstd)",write,0,14.8462
1,"Parquet (pyarrow, zstd)",write,1,13.6425
2,"Parquet (pyarrow, zstd)",write,2,13.6877
3,"Parquet (pyarrow, zstd)",read,0,3.2206
4,"Parquet (pyarrow, zstd)",read,1,2.0124
5,"Parquet (pyarrow, zstd)",read,2,0.9691
6,Lance (lancedb),write,0,6.9939
7,Lance (lancedb),write,1,4.9356
8,Lance (lancedb),write,2,3.5915
9,Lance (lancedb),read,0,4.6634
